In [33]:
import glob, re, os, shutil, random
import pandas as pd
from pathlib import Path
import numpy as np

In [46]:
#############################################
######### Split label file E/I, ... #########
#############################################

TARGET_LIST = ['E/I', 'F/T', 'J/P', 'N/S']
for TARGET in TARGET_LIST:
    df = pd.read_csv("data\\label.csv", index_col= "No.")
    trait = TARGET.split('/')

    os.makedirs(f'data\\{trait[0]}_{trait[1]}\\{trait[0]}', exist_ok=True)
    os.makedirs(f'data\\{trait[0]}_{trait[1]}\\{trait[1]}', exist_ok=True)

    for file in glob.glob("data\\images\\*.jpg"):
            file_id = file.replace('_', '\\').split('\\')[2]
            personality = df.iloc[int(file_id)-1][TARGET].strip(' ')
        
            SOURCE_PATH = f'{file}'
            TARGET_PATH = f'data\\{trait[0]}_{trait[1]}\\{personality}'

            shutil.copy(SOURCE_PATH, TARGET_PATH)

_2.jpg 36
data\N_S\S
data\images\36_3.jpg 36
data\N_S\S
data\images\36_4.jpg 36
data\N_S\S
data\images\36_5.jpg 36
data\N_S\S
data\images\36_6.jpg 36
data\N_S\S
data\images\36_7.jpg 36
data\N_S\S
data\images\370_1.jpg 370
data\N_S\N
data\images\370_2.jpg 370
data\N_S\N
data\images\370_3.jpg 370
data\N_S\N
data\images\370_4.jpg 370
data\N_S\N
data\images\370_5.jpg 370
data\N_S\N
data\images\370_6.jpg 370
data\N_S\N
data\images\370_7.jpg 370
data\N_S\N
data\images\370_8.jpg 370
data\N_S\N
data\images\371_2.jpg 371
data\N_S\N
data\images\371_3.jpg 371
data\N_S\N
data\images\371_5.jpg 371
data\N_S\N
data\images\371_6.jpg 371
data\N_S\N
data\images\371_7.jpg 371
data\N_S\N
data\images\371_9.jpg 371
data\N_S\N
data\images\372_2.jpg 372
data\N_S\N
data\images\372_5.jpg 372
data\N_S\N
data\images\372_6.jpg 372
data\N_S\N
data\images\372_7.jpg 372
data\N_S\N
data\images\372_8.jpg 372
data\N_S\N
data\images\372_9.jpg 372
data\N_S\N
data\images\373_1.jpg 373
data\N_S\N
data\images\373_2.jpg 373
d